In [29]:
import sys

sys.path.append("../")

from sqlmodel import create_engine, Session, select
from chemicalite_orm.models import (
    Compound,
    mol_morgan_bfp,
    mol_from_smiles,
    mol_from_smarts,
)
from chemicalite_orm.molecules import SMILES_SAMPLE as mols
from sqlalchemy import event

In [30]:
DATABASE_URL = "sqlite:///chemicalite.db?enable_load_extension=1"

engine = create_engine(DATABASE_URL, echo=False)


@event.listens_for(engine, "connect")
def load_chemicalite(dbapi_conn, connection_record):
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension("chemicalite")


Compound.__table__.create(engine)

In [31]:
def insert_test_data():
    with Session(engine) as session:
        for i, smiles in enumerate(mols):
            compound = Compound(
                name=f"mol_{i}",
                smiles=smiles,
                molecule=smiles,
                mfp2=mol_morgan_bfp(mol_from_smiles(smiles), 2, 1024),
            )
            session.add(compound)
        session.commit()


insert_test_data()

In [32]:
def test_substructure(smarts="c1ccccc1"):
    query_mol = mol_from_smarts(smarts)
    with Session(engine) as session:
        results = session.exec(
            select(Compound).where(Compound.molecule.hassubstruct(query_mol))
        ).all()
        for r in results:
            print("  -", r.name)


test_substructure()

AttributeError: Neither 'InstrumentedAttribute' object nor 'Comparator' object associated with Compound.molecule has an attribute 'hassubstruct'